In [1]:
import pandas as pd
import numpy as np
import random
import pickle
import itertools
import logging
import re
import time

import matplotlib.pyplot as plt
import seaborn as sns

from konlpy.tag import Okt, Mecab
from gensim.models import Word2Vec, KeyedVectors
from collections import Counter
from tqdm import tqdm
from sklearn import preprocessing

import warnings
warnings.filterwarnings('ignore')

C:\Users\user\.conda\envs\green\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
train = pd.read_csv('./data/train.csv')

In [3]:
train

,index,제출년도,사업명,사업_부처명,계속과제여부,내역사업명,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,요약문_영문키워드,label
0,0,2016,농업기초기반연구,농촌진흥청,신규,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,○ 새로운 해충분류군의 동정기술 개발 및 유입확산 추적,(가) 외래 및 돌발해충의 발생조사 및 종 동정\n\n\n ○ 대상해충 : 최...,"○ 새로운 돌발 및 외래해충의 신속, 정확한 동정법 향상\n\n\n○ 돌발 및 외래...","뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체","nucleotide sequence, molecular marker, species...",24
1,1,2019,이공학학술연구기반구축(R&D),교육부,신규,지역대학우수과학자지원사업(1년~5년),대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...,최종목표: TRAIL 감수성 표적 유전자를 발굴하고 내성제어 기전을 연구. 발굴된...,1차년도\n1) Microarray를 통한 선천적 TRAIL 내성 표적 후보 유전자...,"1) TRAIL 내성 특이적 표적분자를 발굴하고, 이를 이용한 TRAIL 효과 증진...","대장암,항암제 내성,세포사멸,유전자발굴","TRAIL,Colorectal cancer,TRAIL resistance,Apopt...",0
2,2,2016,중소기업기술혁신개발,중소기업청,신규,혁신기업기술개발,비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케어 faci...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n* 소비자 및 바...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n- Tencel/...,* 국내 독자적인 비목질계 셀룰로오스 자원의 파이버 및 부직포 제조 등의 기술 확보...,"기능성 샐룰로오스 파이버,천연섬유,기능성 부직포,뷰티&amp;케어 제품,미용 솜","functional cellulose fiber,natural fiber,funct...",0
3,3,2018,창업성장기술개발(R&D),중소벤처기업부,신규,창업사업화연계과제,소화기 암 진단용 분자영상 형광프로브 개발,# 암특이적 바이오마커 발굴 및 바이오마커에 대한 프로브 개발\n\n\n1) 소화...,# 소화기 암 진단용 분자영상 형광프로브 개발\n\n\n- 국소 도포형 소화기 암 ...,# 암 진단기술의 차별성: 소화기 암 특이 프로브 개발\n\n\n- 최근 체외진단시...,"분자 진단,형광 조영제,프로브,항체,대장암","Molecular diagnosis,Fluorescence,probe,antibod...",0
4,4,2016,이공학개인기초연구지원,교육부,계속,기본연구지원,위암환자의 항암제반응예측을 위한 BRCA 발현검사,"수술이 불가능한 위암환자는 생존기간은 10개월 안팎에 지나지 않고, 항암화학요법에 ...",-In situ hybridization 검사의 정확성을 확인하기 위해 위암세포주 ...,-본 연구는 파라핀보관조직에서 in situ hybridization로 mRNA 및...,"BRCA,제자리부합법,조직미세배열,마이크로RNA,위암,항암제반응,젊은 연령/가족성 위암","BRCA,Insituhybridization,tissuemicroarray,micr...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
174299,174299,2016,개인연구지원,미래창조과학부,신규,중견연구,혈관내막 증식을 치료하기 위한 표적분자 발굴,"증식성 혈관질환과 염증반응과의 관련성이 높으나, 혈관평활근세포의 탈분화를 유도하는 ...",● 1차년: 혈관내막 증식과 관련된 표적분자 발굴 및 염증세포의 활성과 분화에 대한...,심혈관 질환 발생의 새로운 기전 규명뿐만 아니라 제어할 수 있는 새로운 방법을 탐...,"혈관내막 증식,혈관 개조,혈관평활근세포,염증세포,혈관염증,혈관평활근세포 탈분화,표적분자","Intimahyperplasia,Vascularremodeling,Vasculars...",0
174300,174300,2016,개인연구지원,미래창조과학부,신규,중견연구,삼킴곤란 개선을 위한 물성조절 식품시스템에서의 유동층과립 복합소재의 유변물성 변화 ...,삼킴곤란을 겪고 있는 고령자의 안전한 식품섭취는 삼킴에 적합한 유변물성을 가진 물성...,본 연구에서는 여러 가지 유변물성 측정기술을 이용하여 고령자의 삼킴곤란 개선을 위해...,○ 그 동안 연구되지 않은 다양한 물성조절 검복합 소재의 과립제형 및 유변물성 연구...,"삼킴곤란,유변물성,유동층과립,물리적 특성,검복합물","Swallowingdifficulty,Rheologicalproperty,Fludi...",0
174301,174301,2018,이공학학술연구기반구축(R&D),교육부,계속,이공분야학문후속세대양성_대통령POST_DOC펠로우쉽,식물 유전체 정보 고도화 및 핵심 유전자 군 진화연구,"지난 10년여 동안 100여종에 대한 식물 유전체 정보가 완성되어 공개되었지만, 애...",○ 공개된 식물 유전체 정보 및 대상 유전자 군 선발\n\n - 공개된 식물 유...,- 본 과제에서 개발한 annotation tool은 유전체 고도화 부분에서 가장 ...,"유전자 정보 고도화,대상 유전자 군 재분석,비교 유전체 진화 분석,공개 식물 유전체...","Gene model improvement,Target gene family re-a...",0
174302,174302,2019,수요자맞춤형육종자원대량신속발굴기술개발(R&D),농촌진흥청,신규,유용형질 대량 특성평가를 통한 육종모본 발굴,"콩 유전자원 내건성, 내습성 대량평가 체계구축 및 자원선발",1) 내건 및 내습성 유전자원 수집\n\n2) 내건성 자원평가 체계구축\n\n3) ...,[1차 년도(2019)] \n\n1) 내건 및 내습성 유전자원 수집\n\n○ 기존 ...,○ 내건성 및 내습성 특성평가 체계구축 및 우수자원 발굴로 내재해성 콩 품종육성 자...,"콩, 가공적성, 병 저항성, 내재해성, 콩모자이크바이러스","soybean, food application, resistance, biotic-...",23


In [4]:
train = train.fillna('NAN')

In [5]:
train.columns

Index(['index', '제출년도', '사업명', '사업_부처명', '계속과제여부', '내역사업명', '과제명', '요약문_연구목표',
       '요약문_연구내용', '요약문_기대효과', '요약문_한글키워드', '요약문_영문키워드', 'label'],
      dtype='object')

In [6]:
train_1 = train['사업명']
train_2 = train['사업_부처명']
train_3 = train['내역사업명']
train_4 = train['과제명']
train_5 = train['요약문_연구목표']
train_6 = train['요약문_연구내용']
train_7 = train['요약문_기대효과']
train_8 = train['요약문_한글키워드']

In [7]:
train_sent_1 = list(train_1)
train_sent_2 = list(train_2)
train_sent_3 = list(train_3)
train_sent_4 = list(train_4)
train_sent_5 = list(train_5)
train_sent_6 = list(train_6)
train_sent_7 = list(train_7)
train_sent_8 = list(train_8)

In [8]:
len(train_sent_6)

174304

In [9]:
sentences = train_sent_1 + train_sent_2 + train_sent_3 + train_sent_4 + train_sent_5 + train_sent_6 + train_sent_7 + train_sent_8

In [10]:
len(sentences)

1394432

# Tokenize

In [11]:
def text_preprocessing(text_list):
    
    stopwords = ['은','는','이','가']
    tokenizer = Mecab(dicpath=r'C:\mecab\mecab-ko-dic') 
    #tokenizer = Okt()
    token_list = []
    
    for text in tqdm(text_list):
        #text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ]","", text)
        #text = re.sub('[^A-Za-z0-9가-힣]', '', text)
        text = re.sub('[^0-9가-힣]', '', text)
        #text = text.lower()
        token = tokenizer.morphs(text) 
        token = [t for t in token if t not in stopwords or type(t) != float]
        token_list.append(token)
        
    return token_list

In [12]:
tokenized_data = text_preprocessing(sentences)

100%|██████████████████████████████████████████████████████████████████████| 1394432/1394432 [04:28<00:00, 5195.46it/s]


In [14]:
tokenized_data[700000:]

[['발효제',
  '별',
  '약주',
  '의',
  '관능',
  '적',
  '특성',
  '을',
  '도출',
  '하',
  '여',
  '품질',
  '평가',
  '지표',
  '를',
  '개발'],
 ['2',
  '차년',
  '도',
  '는',
  '클라우드',
  '플랫폼',
  '과',
  '연계',
  '된',
  '서비스',
  '프레임워크',
  '와',
  '클라우드',
  '의',
  '기능',
  '고도',
  '화',
  '및',
  '로드',
  '밸',
  '런',
  '싱',
  '클러스터',
  '관리',
  '개발',
  '및',
  '사업',
  '화',
  '주관',
  '기관',
  '1',
  '클라우드',
  '고도',
  '화',
  '개발',
  '2',
  '로드',
  '밸',
  '런',
  '싱',
  '기능',
  '개발',
  '3',
  '클러스터',
  '관리',
  '기능',
  '개발',
  '4',
  '향상',
  '된',
  '손실',
  '복구',
  '개발',
  '수요',
  '처',
  '1',
  '기반',
  '교환기',
  '연동',
  '고도',
  '화',
  '설계',
  '2',
  '기반',
  '교환기',
  '연동',
  '고도',
  '화',
  '에',
  '따른',
  '확장',
  '기능',
  '목표달성',
  '도',
  '지표',
  '1',
  '최대',
  '프레임',
  '60',
  '프레임',
  '달성',
  '최종',
  '목표',
  '60',
  '프레임',
  '2',
  '음성',
  '패킷',
  '손실',
  '복구',
  '15',
  '달성',
  '최종',
  '목표',
  '153720',
  '기준',
  '동시',
  '화',
  '자표',
  '출수',
  '32',
  '자최',
  '종목',
  '표',
  '32',
  '이상',
  '41080',
  '기준',
  '동시',
  '화',
  '

In [15]:
len(tokenized_data)

1394432

In [16]:
type(tokenized_data)

list

In [17]:
# save
with open('./data/tokenized_data.pickle', 'wb') as f:
    pickle.dump(tokenized_data, f, pickle.HIGHEST_PROTOCOL)

# Train

In [ ]:
# load
with open('./tokenized_data.pickle', 'rb') as f:
    tokenized_data = pickle.load(f)

In [20]:
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, epochs=100,  workers=6, sg=1)

2021-07-12 20:32:45,545 : INFO : collecting all words and their counts
2021-07-12 20:32:45,550 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-07-12 20:32:45,576 : INFO : PROGRESS: at sentence #10000, processed 49932 words, keeping 678 word types
2021-07-12 20:32:45,593 : INFO : PROGRESS: at sentence #20000, processed 99925 words, keeping 753 word types
2021-07-12 20:32:45,609 : INFO : PROGRESS: at sentence #30000, processed 149378 words, keeping 793 word types
2021-07-12 20:32:45,627 : INFO : PROGRESS: at sentence #40000, processed 198889 words, keeping 805 word types
2021-07-12 20:32:45,644 : INFO : PROGRESS: at sentence #50000, processed 248490 words, keeping 844 word types
2021-07-12 20:32:45,660 : INFO : PROGRESS: at sentence #60000, processed 298025 words, keeping 861 word types
2021-07-12 20:32:45,676 : INFO : PROGRESS: at sentence #70000, processed 347869 words, keeping 870 word types
2021-07-12 20:32:45,694 : INFO : PROGRESS: at sentence #80000,

2021-07-12 20:32:48,350 : INFO : PROGRESS: at sentence #730000, processed 8297088 words, keeping 58941 word types
2021-07-12 20:32:48,654 : INFO : PROGRESS: at sentence #740000, processed 9441558 words, keeping 63147 word types
2021-07-12 20:32:48,912 : INFO : PROGRESS: at sentence #750000, processed 10591907 words, keeping 66633 word types
2021-07-12 20:32:49,185 : INFO : PROGRESS: at sentence #760000, processed 11726249 words, keeping 69882 word types
2021-07-12 20:32:49,466 : INFO : PROGRESS: at sentence #770000, processed 12863095 words, keeping 72853 word types
2021-07-12 20:32:49,792 : INFO : PROGRESS: at sentence #780000, processed 13991261 words, keeping 75469 word types
2021-07-12 20:32:50,147 : INFO : PROGRESS: at sentence #790000, processed 15123661 words, keeping 77822 word types
2021-07-12 20:32:50,481 : INFO : PROGRESS: at sentence #800000, processed 16273503 words, keeping 80058 word types
2021-07-12 20:32:50,790 : INFO : PROGRESS: at sentence #810000, processed 17408369

2021-07-12 20:33:11,739 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 93169803 word corpus (100.0%% of original 93169803, drops 0)', 'datetime': '2021-07-12T20:33:11.739013', 'gensim': '4.0.1', 'python': '3.7.10 (default, Feb 26 2021, 13:06:18) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'prepare_vocab'}
2021-07-12 20:33:12,647 : INFO : deleting the raw counts dictionary of 167634 items
2021-07-12 20:33:12,654 : INFO : sample=0.001 downsamples 50 most-common words
2021-07-12 20:33:12,655 : INFO : Word2Vec lifecycle event {'msg': 'downsampling leaves estimated 74058369.21791801 word corpus (79.5%% of prior 93169803)', 'datetime': '2021-07-12T20:33:12.655558', 'gensim': '4.0.1', 'python': '3.7.10 (default, Feb 26 2021, 13:06:18) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'prepare_vocab'}
2021-07-12 20:33:14,094 : INFO : estimated required memory for 167634 words and 100 dimensions: 21792420

2021-07-12 20:34:12,833 : INFO : EPOCH 1 - PROGRESS: at 71.43% examples, 716405 words/s, in_qsize 11, out_qsize 0
2021-07-12 20:34:13,841 : INFO : EPOCH 1 - PROGRESS: at 71.71% examples, 715974 words/s, in_qsize 12, out_qsize 0
2021-07-12 20:34:14,846 : INFO : EPOCH 1 - PROGRESS: at 72.00% examples, 716416 words/s, in_qsize 11, out_qsize 0
2021-07-12 20:34:15,859 : INFO : EPOCH 1 - PROGRESS: at 72.28% examples, 716112 words/s, in_qsize 11, out_qsize 0
2021-07-12 20:34:16,859 : INFO : EPOCH 1 - PROGRESS: at 72.57% examples, 716090 words/s, in_qsize 11, out_qsize 0
2021-07-12 20:34:17,868 : INFO : EPOCH 1 - PROGRESS: at 72.84% examples, 715606 words/s, in_qsize 11, out_qsize 0
2021-07-12 20:34:18,875 : INFO : EPOCH 1 - PROGRESS: at 73.12% examples, 715449 words/s, in_qsize 11, out_qsize 0
2021-07-12 20:34:19,888 : INFO : EPOCH 1 - PROGRESS: at 73.41% examples, 715202 words/s, in_qsize 11, out_qsize 0
2021-07-12 20:34:20,892 : INFO : EPOCH 1 - PROGRESS: at 73.68% examples, 714648 words/s,

2021-07-12 20:35:19,999 : INFO : EPOCH 2 - PROGRESS: at 59.88% examples, 758923 words/s, in_qsize 11, out_qsize 0
2021-07-12 20:35:21,000 : INFO : EPOCH 2 - PROGRESS: at 60.47% examples, 757511 words/s, in_qsize 11, out_qsize 0
2021-07-12 20:35:22,006 : INFO : EPOCH 2 - PROGRESS: at 61.05% examples, 755671 words/s, in_qsize 11, out_qsize 0
2021-07-12 20:35:23,015 : INFO : EPOCH 2 - PROGRESS: at 61.65% examples, 754474 words/s, in_qsize 11, out_qsize 0
2021-07-12 20:35:24,020 : INFO : EPOCH 2 - PROGRESS: at 62.22% examples, 752593 words/s, in_qsize 11, out_qsize 0
2021-07-12 20:35:25,022 : INFO : EPOCH 2 - PROGRESS: at 62.64% examples, 750299 words/s, in_qsize 11, out_qsize 0
2021-07-12 20:35:26,038 : INFO : EPOCH 2 - PROGRESS: at 62.90% examples, 746081 words/s, in_qsize 11, out_qsize 0
2021-07-12 20:35:27,055 : INFO : EPOCH 2 - PROGRESS: at 63.06% examples, 733126 words/s, in_qsize 11, out_qsize 0
2021-07-12 20:35:28,081 : INFO : EPOCH 2 - PROGRESS: at 63.17% examples, 717169 words/s,

...

2021-07-12 23:07:23,951 : INFO : EPOCH 98 - PROGRESS: at 56.67% examples, 861170 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:07:24,957 : INFO : EPOCH 98 - PROGRESS: at 57.30% examples, 855512 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:07:25,959 : INFO : EPOCH 98 - PROGRESS: at 57.93% examples, 850742 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:07:26,960 : INFO : EPOCH 98 - PROGRESS: at 58.57% examples, 846977 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:07:27,969 : INFO : EPOCH 98 - PROGRESS: at 59.20% examples, 842979 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:07:28,976 : INFO : EPOCH 98 - PROGRESS: at 59.84% examples, 839635 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:07:29,977 : INFO : EPOCH 98 - PROGRESS: at 60.47% examples, 836653 words/s, in_qsize 12, out_qsize 0
2021-07-12 23:07:30,980 : INFO : EPOCH 98 - PROGRESS: at 61.10% examples, 833241 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:07:31,982 : INFO : EPOCH 98 - PROGRESS: at 61.73% examples, 831050

2021-07-12 23:08:36,653 : INFO : EPOCH 98 - PROGRESS: at 85.11% examples, 783648 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:08:37,664 : INFO : EPOCH 98 - PROGRESS: at 85.57% examples, 783545 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:08:38,678 : INFO : EPOCH 98 - PROGRESS: at 86.04% examples, 783406 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:08:39,691 : INFO : EPOCH 98 - PROGRESS: at 86.50% examples, 783304 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:08:40,702 : INFO : EPOCH 98 - PROGRESS: at 86.96% examples, 783117 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:08:41,720 : INFO : EPOCH 98 - PROGRESS: at 87.42% examples, 783027 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:08:42,728 : INFO : EPOCH 98 - PROGRESS: at 92.20% examples, 784133 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:08:43,735 : INFO : EPOCH 98 - PROGRESS: at 98.00% examples, 785497 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:08:44,020 : INFO : worker thread finished; awaiting finish of 5 mo

2021-07-12 23:09:43,565 : INFO : EPOCH 99 - PROGRESS: at 73.48% examples, 791612 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:09:44,565 : INFO : EPOCH 99 - PROGRESS: at 73.79% examples, 791346 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:09:45,566 : INFO : EPOCH 99 - PROGRESS: at 74.09% examples, 790938 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:09:46,580 : INFO : EPOCH 99 - PROGRESS: at 74.39% examples, 790526 words/s, in_qsize 12, out_qsize 0
2021-07-12 23:09:47,582 : INFO : EPOCH 99 - PROGRESS: at 74.69% examples, 790191 words/s, in_qsize 11, out_qsize 1
2021-07-12 23:09:48,585 : INFO : EPOCH 99 - PROGRESS: at 75.00% examples, 790124 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:09:49,607 : INFO : EPOCH 99 - PROGRESS: at 75.47% examples, 789627 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:09:50,616 : INFO : EPOCH 99 - PROGRESS: at 75.93% examples, 789384 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:09:51,625 : INFO : EPOCH 99 - PROGRESS: at 76.39% examples, 789051

2021-07-12 23:10:49,513 : INFO : EPOCH 100 - PROGRESS: at 64.79% examples, 804542 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:10:50,521 : INFO : EPOCH 100 - PROGRESS: at 65.09% examples, 802715 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:10:51,535 : INFO : EPOCH 100 - PROGRESS: at 65.39% examples, 801682 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:10:52,536 : INFO : EPOCH 100 - PROGRESS: at 65.70% examples, 800744 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:10:53,543 : INFO : EPOCH 100 - PROGRESS: at 65.99% examples, 799630 words/s, in_qsize 12, out_qsize 0
2021-07-12 23:10:54,548 : INFO : EPOCH 100 - PROGRESS: at 66.30% examples, 798710 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:10:55,554 : INFO : EPOCH 100 - PROGRESS: at 66.60% examples, 797421 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:10:56,554 : INFO : EPOCH 100 - PROGRESS: at 66.90% examples, 796503 words/s, in_qsize 11, out_qsize 0
2021-07-12 23:10:57,571 : INFO : EPOCH 100 - PROGRESS: at 67.19% example

2021-07-12 23:11:53,545 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec(vocab=167634, vector_size=100, alpha=0.025)', 'datetime': '2021-07-12T23:11:53.545265', 'gensim': '4.0.1', 'python': '3.7.10 (default, Feb 26 2021, 13:06:18) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}


In [21]:
model.save('./data/word2vec_mecab.model')

2021-07-12 23:11:53,647 : INFO : Word2Vec lifecycle event {'fname_or_handle': './data/word2vec_mecab.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-07-12T23:11:53.647618', 'gensim': '4.0.1', 'python': '3.7.10 (default, Feb 26 2021, 13:06:18) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'saving'}
2021-07-12 23:11:53,672 : INFO : storing np array 'vectors' to ./data/word2vec_mecab.model.wv.vectors.npy
2021-07-12 23:11:53,871 : INFO : storing np array 'syn1neg' to ./data/word2vec_mecab.model.syn1neg.npy
2021-07-12 23:11:54,070 : INFO : not storing attribute cum_table
2021-07-12 23:12:05,555 : INFO : saved ./data/word2vec_mecab.model


In [22]:
word_vectors = model.wv
word_vectors

In [24]:
word_vectors['유전']

array([ 0.04640033,  0.37383494,  0.4777076 , -0.8934443 , -0.28223076,
        0.02873539,  0.04378541, -0.062386  , -0.01374035, -0.6614364 ,
        0.1107081 , -0.00584422, -0.2200083 ,  0.13872954,  0.00799854,
       -0.36598244,  0.06146726, -0.06398597, -0.08076894,  0.6220358 ,
        0.09618143,  0.1651417 , -0.24178503,  0.03386214, -0.28659916,
       -0.36489725,  0.29756364, -0.37432894,  0.10974748,  0.06738941,
        0.165629  ,  0.1398579 ,  0.12482858,  0.29655498,  0.13712554,
        0.01515673,  0.54028815,  0.34583187,  0.14104876, -0.29464734,
       -0.13605246, -0.11590624, -0.27429894, -0.43549263, -0.14098395,
        0.21422155, -0.2460066 ,  0.02029849, -0.13706759, -0.33366072,
        0.45131776, -0.4205082 ,  0.34613323, -0.0175306 ,  0.18838125,
       -0.47283813,  0.73532677, -0.04997841, -0.14034389,  0.718693  ,
        0.08439554,  0.2267647 ,  0.44171464, -0.3030995 , -0.3380128 ,
        0.09274893,  0.23120646, -0.16359666, -0.02182421, -0.28

In [ ]:
vocabs = word_vectors.vocab.keys()
vocabs

In [ ]:
word_vectors_list = [word_vectors[v] for v in vocabs]
word_vectors_list

In [25]:
model_result1 = model.wv.most_similar("유전")
print(model_result1)

[('변이', 0.7798404097557068), ('후성', 0.770308792591095), ('유전학', 0.7314926385879517), ('유전자', 0.6997476816177368), ('유전체', 0.6900604963302612), ('다형성', 0.6755560040473938), ('유전자형', 0.6484876275062561), ('돌연변이', 0.6415858268737793), ('단일염기', 0.6208983659744263), ('변이형', 0.6186596155166626)]


In [ ]:
word_vectors.similarity(w1 = '블록체인', w2 = '산업혁명')

# Inference

In [ ]:
model = Word2Vec.load('./data/embedding/word2vec_okt.model')

In [ ]:
word_vectors = model.wv
word_vectors

In [ ]:
word_vectors['딥러닝']

In [ ]:
vocabs = word_vectors.vocab.keys()
vocabs

In [ ]:
word_vectors_list = [word_vectors[v] for v in vocabs]
word_vectors_list

In [ ]:
model_result1 = model.wv.most_similar("인공지능")
print(model_result1)

In [ ]:
word_vectors.similarity(w1 = '블록체인', w2 = '산업혁명')

In [ ]:
word_vectors['본']

In [ ]:
word_vectors['논문에서는']

In [ ]:
word_vectors['최신']

In [ ]:
word_vectors['모델을']

In [ ]:
word_vectors['개발']

In [ ]:
word_vectors['하였다']